In [ ]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
import requests
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm

#img
import cv2

In [ ]:
#brd_idx, prd_idx 정렬한 dataframe 만들기

idx = 0
for i in tqdm(fnl_df_brd_w_prd.index):
    for j in range(len(fnl_df_brd_w_prd.loc[i, 'product_name'])):
        prd = {"brd_idx": i,
               "prd_idx": j,
              "prd_id": fnl_df_brd_w_prd.loc[i, 'product_link'][j].split('/')[-1],
              "brd_name": fnl_df_brd_w_prd.loc[i, 'brand_name'],
              "prd_name": fnl_df_brd_w_prd.loc[i, 'product_name'][j]}
        df_idx_brd_prd.loc[idx] = prd
        idx += 1

In [ ]:
fnl_df_glowpick_prd_info_w_img_src_drop_naver = pd.read_csv("fnl_df_glowpick_prd_info_w_img_src_drop_naver.csv")

df_idx_brd_prd = df_idx_brd_prd.drop(['brd_name', 'prd_name'], axis='columns')

fnl_df_glowpick_prd_info_w_img_src_drop_naver = pd.merge(df_idx_brd_prd, fnl_df_glowpick_prd_info_w_img_src_drop_naver, how='inner', left_on='prd_id', right_on='제품 id')

drop_columns_lst = ['제품 용량', '제품 가격', '유사 제품군 랭킹(int)', '유사 제품군 랭킹(text)',
       '브랜드 내 랭킹(int)', '제품 수상명', '제품 수상 부문', '제품 수상 순위', '제품 설명', '제품 컬러/타입',
       '제품 판매처', '제품 성분구성: 위험도 미정 (갯수)', '제품 성분구성: 위험도 미정 (성분(한글))',
       '제품 성분구성: 위험도 미정 (성분(영어))', '제품 성분구성: 위험도 미정 (성분(설명))',
       '제품 성분구성: 낮은 위험도 (갯수)', '제품 성분구성: 낮은 위험도 (성분(한글))',
       '제품 성분구성: 낮은 위험도 (성분(영어))', '제품 성분구성: 낮은 위험도 (성분(설명))',
       '제품 성분구성: 중간 위험도 (갯수)', '제품 성분구성: 중간 위험도 (성분(한글))',
       '제품 성분구성: 중간 위험도(성분(영어))', '제품 성분구성: 중간 위험도 (성분(설명))',
       '제품 성분구성: 높은 위험도 (갯수)', '제품 성분구성: 높은 위험도 (성분(한글))',
       '제품 성분구성: 높은 위험도 (성분(영어))', '제품 성분구성: 높은 위험도 (성분(설명))', '제품 카테고리', '리뷰',
       '별점', '리뷰 일자', '사용자 리뷰 개수', '사용자 피부 타입', '사용자 나이', '사용자 성별', '사용자 이름']
       
glowpick_prd_img_src = fnl_df_glowpick_prd_info_w_img_src_drop_naver.drop(drop_columns_lst, axis='columns')

glowpick_prd_img_src.to_csv("glowpick_prd_img_src.csv", index=False)

In [ ]:
glowpick_prd_img_src = pd.read_csv("glowpick_prd_img_src.csv")

---

In [ ]:
def get_img(img_src, brd_idx, prd_idx):
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument(f'user-agent={userAgent}')
    wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    wd.get(img_src)
    time.sleep(5)
    
    socket.setdefaulttimeout(30)
    
    while True:

        try:
            #wd.find_element_by_xpath('//*[@id=\"default-layout\"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
            #time.sleep(5)

            #html = wd.page_source
            #bs = BeautifulSoup(html, 'html.parser')

            #images = bs.select('.image__img')
            #time.sleep(3)

            if not os.path.exists('glowpick_img/brand_{}'.format(brd_idx)): 
                os.makedirs('glowpick_img/brand_{}'.format(brd_idx))
            
            #이미지가 gif이면 frame 하나만 가져오기
            if str(img_src).lower().endswith('.gif?w=500'):
                gif = cv2.VideoCapture(img_src)
                ret, frame = gif.read()  # ret=True if it finds a frame else False.
                while ret:
                    rate_num = 500 / frame.shape[1]
                    width = 500
                    height = int(frame.shape[0] * rate_num)
                    frame = cv2.resize(frame, dsize=(width, height), interpolation=cv2.INTER_AREA)
                    cv2.imwrite('glowpick_img/brand_{}/brand_{}_{}.png'.format(brd_idx, brd_idx, prd_idx), frame)
                    return None
            else:
                #img_src = wd.find_element_by_xpath("/html/body/div/div/div/div/main/div/section/div[1]/div/div/img").get_attribute('src')
                #img_src = re.sub("w=[0-9]*", "w=500", img_src)
                img_nparray = np.asarray(bytearray(requests.get(img_src).content), dtype=np.uint8)
                crawl_img = cv2.imdecode(img_nparray, cv2.IMREAD_COLOR)
                cv2.imwrite('glowpick_img/brand_{}/brand_{}_{}.png'.format(brd_idx, brd_idx, prd_idx), crawl_img)
                wd.close()
                return None
        
        except urllib3.exceptions.ReadTimeoutError:
            time.sleep(5)

        except Exception as e:
            print("에러가 발생했습니다.", e)
            missing_img.append([brd_idx, prd_idx, img_src])
            wd.close()
            return None

In [ ]:
missing_img = []

In [ ]:
for i in tqdm(glowpick_prd_img_src.loc[(pd.isnull(glowpick_prd_img_src['이미지 src']) == False) & (pd.isnull(glowpick_prd_img_src['img_complete']) == True)][730:].index):
    brd_idx = glowpick_prd_img_src.loc[i, 'brd_idx']
    prd_idx = glowpick_prd_img_src.loc[i, 'prd_idx']
    img_src = glowpick_prd_img_src.loc[i, '이미지 src']
    get_img(img_src, brd_idx, prd_idx)

---
이미지 수집 완료된 brd_idx, prd_idx 파악, 원df에 check

In [ ]:
import os
cwd = os.getcwd()

In [ ]:
from os import listdir

path_glowpick_img= cwd + '/glowpick_img'
path_glowpick_img_brand = []

for x in listdir(path_glowpick_img):
    path_glowpick_img_brand.append(cwd + '/glowpick_img/{}'.format(x))

In [ ]:
path_glowpick_img_brand.remove(cwd + 'glowpick_img/.DS_Store')

In [ ]:
complete_lst = pd.DataFrame(columns = ['brd_idx', 'prd_idx_lst'])

idx = 0

for i in path_glowpick_img_brand:
    complete_brd_idx = int(i.split('_')[-1])
    complete_prd_idx_lst = [int(x.split('_')[-1].split('.')[0]) for x in listdir(i)]
    
    complete_lst.loc[idx] = {
        'brd_idx': complete_brd_idx,
        'prd_idx_lst': complete_prd_idx_lst
    }
    idx += 1

In [ ]:
#원dataframe인 glowpick_prd_img_src에 표시

for i in tqdm(complete_lst.index):
    brd_idx = complete_lst.loc[i, 'brd_idx']
    for j in complete_lst.loc[i, 'prd_idx_lst']:
        prd_idx = j
        glowpick_prd_img_src.loc[(glowpick_prd_img_src['brd_idx'] == brd_idx) & (glowpick_prd_img_src['prd_idx'] == prd_idx), 'img_complete'] = 1

---
resize image

: width 500으로 맞추기

In [ ]:
from PIL import Image
from tqdm.auto import tqdm

for i in tqdm(path_glowpick_img_brand):
    try:
        for j in listdir(i):
            im = Image.open(i +'/' + j)
            image_size = im.size
            if image_size[0] != 500:
                rate = 500 / image_size[0]
                width = 500
                height = image_size[1] * rate
                im = im.resize((int(width), int(height)))
                im.save(i +'/' + j)
                print(i +'/' + j)
    except :
        print(i)
        pass